In [ ]:
import os

import numpy as np
import torch
import accelerate

from rd3d.datasets import build_dataloader
from rd3d.core import Config
from rd3d import PROJECT_ROOT

os.chdir(PROJECT_ROOT)
acc = accelerate.Accelerator()
cfg = Config.fromfile_py("configs/voxformer/voxformer_4x2_80e_kitti_3cls.py")
dataloader = build_dataloader(cfg.DATASET, cfg.RUN)

In [42]:
from rd3d.models import build_detector
from tqdm import tqdm
from rd3d.utils import viz_utils
from matplotlib import pyplot as plt
from collections import defaultdict

model = build_detector(cfg.MODEL, dataset=dataloader.dataset).cuda()

map_num = []
for batch_dict in tqdm(iterable=dataloader):
    dataloader.dataset.load_data_to_gpu(batch_dict)
    batch_dict = model.vfe(batch_dict)
    vox_numbs = batch_dict['voxel_numbers']
    vox_coors = batch_dict['voxel_coords']
    vox_feats = batch_dict['voxel_features']
    ind, vox_nums = model.backbone_3d.mapping(vox_numbs, vox_coors)
    ind1, ind2, ind12, ind21 = ind
    vox_coors = vox_coors[ind1]
    gs = model.backbone_3d.group_size
    mp = torch.div(ind21.view(-1, gs), 32, rounding_mode='trunc').tolist()
    for imp in mp:
        map_num.append(len(set(imp)))
print(np.mean(map_num))
# """
# flip_coors: 29.8
# reverse_coors: 29.8
# shift: 29.76
# """

100%|██████████| 464/464 [00:23<00:00, 19.37it/s]


3.1545148000253533


In [ ]:

 # vox_feats = vox_feats[ind1]
    # vox_coors = vox_coors[ind1]
    # points = vox_feats[vox_coors[:, 0] == 0, :3]
    # ind = torch.randperm(points.shape[0]//gs)[:, None].repeat(1, gs).view(-1)
    # ind = ind / ind.max()
    # colors = plt.get_cmap('tab20c')(ind)[:, :3]
    # viz_utils.viz_scene((points, colors))
    # assert ((vox_feats[ind1] == vox_feats[ind2][ind21]).all())
    # print(vox_feats[ind1])
    # print(vox_feats[ind2][ind21])
    # vox_feats = vox_feats[ind2][ind21]
    # vox_coors = vox_coors[ind2][ind21]
    # points = vox_feats[vox_coors[:, 0] == 0, :3]
    # ind = torch.randperm(points.shape[0]//gs)[:, None].repeat(1, gs).view(-1)
    # ind = ind / ind.max()
    # colors = plt.get_cmap('tab20c')(ind)[:, :3]
    # viz_utils.viz_scene((points, colors))

[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
